In [ ]:
import os
import random
import pandas as pd
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class UCF101Dataset(Dataset):
    def __init__(self, csv_file, video_base_path, selected_labels=None, 
                 num_frames_per_video=16, resize=(112, 112), transform=None):
        
        self.video_base_path = video_base_path
        self.num_frames_per_video = num_frames_per_video
        self.resize = resize
        
        # Load CSV and filter by selected labels if provided
        self.df = pd.read_csv(csv_file)
        if selected_labels is not None:
            self.df = self.df[self.df['label'].isin(selected_labels)].reset_index(drop=True)
        
        self.transform = transform
        
        # Keep a list of valid videos to avoid retrying problematic ones
        self.valid_indices = list(range(len(self.df)))
        self.problematic_videos = set()
        
    def __len__(self):
        return len(self.valid_indices)
    
    def extract_frames(self, video_path):
        """Extract frames from a video with improved error handling"""
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video {video_path}")
                return None

            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total_frames <= 0:
                print(f"Error: Video {video_path} has no frames")
                cap.release()
                return None
                
            # For videos with fewer frames than requested, use uniform sampling with repetition
            if total_frames < self.num_frames_per_video:
                print(f"Warning: Video {video_path} has only {total_frames} frames, using uniform sampling with repetition")
                frame_indices = np.linspace(0, total_frames - 1, self.num_frames_per_video, dtype=int)
            else:
                # Use uniform sampling for videos with sufficient frames
                frame_indices = np.linspace(0, total_frames - 1, self.num_frames_per_video, dtype=int)
            
            frames = []
            # Maximum retries for frame reading
            max_retries = 3

            for idx in frame_indices:
                success = False
                retry_count = 0
                
                # Try a few times to read the frame at the exact index
                while not success and retry_count < max_retries:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                    ret, frame = cap.read()
                    if ret:
                        success = True
                        break
                    retry_count += 1
                
                # If still failing, try to read nearby frames instead
                if not success:
                    # Try frames before and after the desired index
                    for offset in range(1, min(10, total_frames)):
                        # Try frame before
                        if idx - offset >= 0:
                            cap.set(cv2.CAP_PROP_POS_FRAMES, idx - offset)
                            ret, frame = cap.read()
                            if ret:
                                success = True
                                break
                        
                        # Try frame after
                        if idx + offset < total_frames:
                            cap.set(cv2.CAP_PROP_POS_FRAMES, idx + offset)
                            ret, frame = cap.read()
                            if ret:
                                success = True
                                break
                
                # If we still couldn't read a frame, use a black frame as fallback
                if not success:
                    print(f"Warning: Could not read frame {idx} from {video_path}, using black frame")
                    frame = np.zeros((self.resize[0], self.resize[1], 3), dtype=np.uint8)
                
                # Process the frame
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, self.resize)
                frames.append(frame)
            
            cap.release()

            if len(frames) != self.num_frames_per_video:
                print(f"Error: Expected {self.num_frames_per_video} frames but got {len(frames)} from {video_path}")
                return None

            frames_np = np.stack(frames).astype(np.float32) / 255.0  # shape (num_frames, H, W, C)

            # Convert to tensor and permute to (num_frames, C, H, W)
            frames_tensor = torch.from_numpy(frames_np).permute(0, 3, 1, 2)

            if self.transform:
                frames_tensor = self.transform(frames_tensor)
            
            return frames_tensor
            
        except Exception as e:
            print(f"Exception processing video {video_path}: {e}")
            return None
    
    def __getitem__(self, idx):
        """Get an item with better error handling for problem videos"""
        try:
            # Map the external index to our valid indices
            valid_idx = self.valid_indices[idx]
            row = self.df.iloc[valid_idx]
            clip_rel_path = row['clip_path'].lstrip("/")
            video_path = os.path.join(self.video_base_path, clip_rel_path)
            
            if not os.path.exists(video_path):
                raise FileNotFoundError(f"Video file not found: {video_path}")
            
            frames = self.extract_frames(video_path)
            if frames is None:
                # Add to problematic videos and try a new one
                self.problematic_videos.add(valid_idx)
                # Find the next valid index that's not in problematic_videos
                remaining_videos = set(self.valid_indices) - self.problematic_videos
                if not remaining_videos:
                    # Reset if we've marked all videos as problematic
                    print("Warning: All videos marked as problematic. Resetting problematic videos list.")
                    self.problematic_videos = set()
                    remaining_videos = set(self.valid_indices)
                
                new_idx = random.choice(list(remaining_videos))
                # Find the position of new_idx in valid_indices
                new_pos = self.valid_indices.index(new_idx)
                return self.__getitem__(new_pos)
            
            label = row['label']
            return frames, label
            
        except Exception as e:
            print(f"Error in __getitem__ with idx {idx}: {e}")
            # Try the next index
            new_idx = (idx + 1) % len(self)
            return self.__getitem__(new_idx)


# Map string labels to integer indices
label_to_index = {
    'PlayingCello': 0,
    'PlayingDhol': 1,
    'TennisSwing': 2,
    'CricketShot': 3,
    'HorseRiding': 4
}

# Custom wrapper to convert string labels to integer indices
def collate_fn(batch):
    frames, labels = zip(*batch)
    frames = torch.stack(frames)
    labels_in = torch.tensor([label_to_index[label] for label in labels], dtype=torch.long)
    return frames, labels_in

# Optional: normalization transform
transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225])

# Create training dataset
train_dataset = UCF101Dataset(
    csv_file="train_updated.csv",
    video_base_path="data",
    selected_labels=['PlayingCello','PlayingDhol','TennisSwing','CricketShot','HorseRiding'],
    num_frames_per_video=16,
    resize=(112, 112),
    transform=transform
)

# Create validation dataset
val_dataset = UCF101Dataset(
    csv_file="val_updated.csv",
    video_base_path="data",
    selected_labels=['PlayingCello','PlayingDhol','TennisSwing','CricketShot','HorseRiding'],
    num_frames_per_video=16,
    resize=(112, 112),
    transform=transform
)

from torch.utils.data import DataLoader

# Create data loaders with prefetching and num_workers
train_loader = DataLoader(
    train_dataset, 
    batch_size=5, 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=0,  # Increase if you have multiple CPU cores
    pin_memory=torch.cuda.is_available(),  # Speed up transfer to GPU
    persistent_workers=False  # Set to True if using num_workers > 0
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=5, 
    shuffle=False, 
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=torch.cuda.is_available(),
    persistent_workers=False
)

# The rest of your C3D model and training code remains the same
import torch
import torch.nn as nn

class C3D(nn.Module):
    def __init__(self, num_classes=101):
        super(C3D, self).__init__()

        self.features = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2)),

            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)),

            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)),

            nn.Conv3d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)),

            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        )

        # Dynamically determine the flattened size
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, 16, 112, 112)
            dummy_output = self.features(dummy_input)
            self.flattened_size = dummy_output.view(1, -1).size(1)

        self.classifier = nn.Sequential(
            nn.Linear(self.flattened_size, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.reshape(x.size(0), -1)
        return self.classifier(x)

import torch.optim as optim
from datetime import datetime

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, device="cuda"):
    """Train the model with detailed logging and error handling"""
    model = model.to(device)
    start_time = datetime.now()
    training_history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    for epoch in range(num_epochs):
        epoch_start = datetime.now()
        print(f"Starting Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        batch_count = 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.permute(0, 2, 1, 3, 4)  # Convert [B, T, C, H, W] to [B, C, T, H, W]
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                # Update statistics
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                batch_size = labels.size(0)
                total += batch_size
                correct += (predicted == labels).sum().item()
                batch_count += 1
                
                # Print batch progress every 10 batches
                if (batch_idx + 1) % 10 == 0:
                    current_acc = 100 * correct / total
                    print(f"Epoch {epoch+1}/{num_epochs} - Batch {batch_idx+1}/{len(train_loader)} - "
                          f"Loss: {total_loss/batch_count:.4f} - Acc: {current_acc:.2f}%")
                    
            except Exception as e:
                print(f"Error in training batch {batch_idx}: {e}")
                continue
        
        # Calculate training metrics
        train_loss = total_loss / max(1, batch_count)
        train_acc = 100 * correct / max(1, total)
        training_history['train_loss'].append(train_loss)
        training_history['train_acc'].append(train_acc)
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        val_batch_count = 0
        
        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(val_loader):
                try:
                    inputs, labels = inputs.to(device), labels.to(device)
                    inputs = inputs.permute(0, 2, 1, 3, 4)
                    
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    batch_size = labels.size(0)
                    val_total += batch_size
                    val_correct += (predicted == labels).sum().item()
                    val_batch_count += 1
                    
                except Exception as e:
                    print(f"Error in validation batch {batch_idx}: {e}")
                    continue
        
        # Calculate validation metrics
        val_loss = val_loss / max(1, val_batch_count)
        val_acc = 100 * val_correct / max(1, val_total)
        training_history['val_loss'].append(val_loss)
        training_history['val_acc'].append(val_acc)
        
        # Print epoch summary
        epoch_time = datetime.now() - epoch_start
        print(f"Epoch {epoch+1}/{num_epochs} Summary:")
        print(f"Training   - Loss: {train_loss:.4f} - Accuracy: {train_acc:.2f}%")
        print(f"Validation - Loss: {val_loss:.4f} - Accuracy: {val_acc:.2f}%")
        print(f"Epoch completed in {epoch_time}")
        print("-" * 60)
        
        # Save checkpoint (optional - uncomment if needed)
        # if (epoch + 1) % 5 == 0:
        #     torch.save({
        #         'epoch': epoch,
        #         'model_state_dict': model.state_dict(),
        #         'optimizer_state_dict': optimizer.state_dict(),
        #         'train_loss': train_loss,
        #         'val_loss': val_loss,
        #     }, f'checkpoint_epoch_{epoch+1}.pt')
    
    total_time = datetime.now() - start_time
    print(f"Training completed in {total_time}")
    return model, training_history

# Set up the model, criterion, and optimizer
num_classes = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = C3D(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=5,
    device=device
)

print("Training complete!")

Using device: cuda
Starting Epoch 1/5
Epoch 1/5 - Batch 10/124 - Loss: 10.1500 - Acc: 12.00%
Epoch 1/5 - Batch 20/124 - Loss: 5.8897 - Acc: 17.00%
Epoch 1/5 - Batch 30/124 - Loss: 4.4634 - Acc: 20.67%
Epoch 1/5 - Batch 40/124 - Loss: 3.7535 - Acc: 21.00%
Epoch 1/5 - Batch 50/124 - Loss: 3.3258 - Acc: 20.80%
Epoch 1/5 - Batch 60/124 - Loss: 3.0393 - Acc: 21.00%
Epoch 1/5 - Batch 70/124 - Loss: 2.8355 - Acc: 20.86%
Epoch 1/5 - Batch 80/124 - Loss: 2.6853 - Acc: 20.50%
Epoch 1/5 - Batch 90/124 - Loss: 2.5667 - Acc: 19.78%
Epoch 1/5 - Batch 100/124 - Loss: 2.4716 - Acc: 19.60%
Epoch 1/5 - Batch 110/124 - Loss: 2.3936 - Acc: 19.82%
Epoch 1/5 - Batch 120/124 - Loss: 2.3286 - Acc: 19.50%
Epoch 1/5 Summary:
Training   - Loss: 2.3052 - Accuracy: 19.58%
Validation - Loss: 1.6102 - Accuracy: 19.61%
Epoch completed in 0:01:18.316027
------------------------------------------------------------
Starting Epoch 2/5
Epoch 2/5 - Batch 10/124 - Loss: 1.6069 - Acc: 22.00%
Epoch 2/5 - Batch 20/124 - Loss: 